# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
cities = "../output_data/cities.csv"

# read file into data frame
city_weather = pd.read_csv(cities)
city_weather.head()


,Unnamed: 0,City,Latitude,Longitude,Temperature,% Humidity,% Cloudiness,Wind Speed (mph)
0,0,umm lajj,25.0213,37.2685,75.40,61,0,6.44
1,1,atuona,-9.8000,-139.0333,79.03,78,29,13.44
2,2,busselton,-33.6500,115.3333,69.33,53,0,10.65
3,3,half moon bay,37.4636,-122.4286,49.50,62,0,28.77
4,4,lincoln,40.8000,-96.6670,55.67,43,2,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [17]:
# bring in location data pairs, humidity data, define map and layer, and add layer to map

# store latitude and longitude in locations
locations = city_weather[["Latitude", "Longitude"]]
humidity= city_weather["% Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# add layer
fig.add_layer(heat_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# reduce data frame to cities with temps in between 75 and 85 degrees F, with cloudiness less than 10% and Humidity less than 50%
vacation_weather = city_weather[(city_weather["Temperature"] < 85) & (city_weather["Temperature"] > 75) & (city_weather["% Cloudiness"] <10) & (city_weather["% Humidity"] <50)]
# drop null values
vacation_weather.dropna(inplace = True) 
vacation_weather

/Users/taylorbohl/opt/anaconda3/envs/PythonData38/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Unnamed: 0,City,Latitude,Longitude,Temperature,% Humidity,% Cloudiness,Wind Speed (mph)
25,25,abilene,32.4487,-99.7331,76.39,21,0,4.61
54,54,sur,22.5667,59.5289,83.01,36,0,4.00
74,74,agadez,19.7500,10.2500,81.97,7,0,6.73
192,192,arlit,18.7369,7.3853,83.79,8,7,5.03
223,223,tanout,14.9709,8.8879,80.74,9,1,7.36
271,271,kidal,18.4411,1.4078,82.71,5,0,10.16
315,315,mecca,21.4267,39.8261,79.66,33,0,1.21
349,349,gunnedah,-30.9833,150.2500,77.74,48,5,7.81
466,466,nizwa,22.9333,57.5333,78.46,12,0,6.35
540,540,sao desiderio,-12.3633,-44.9733,75.06,46,7,2.39


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# Create dataframe and empty column
hotel_df = vacation_weather[["City"]]
hotel_df["Hotel Name"] = ""
hotel_df

/var/folders/h1/xb5j3d_136j05hmpd9xpx6580000gn/T/ipykernel_46092/264571290.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Hotel Name
25,abilene,
54,sur,
74,agadez,
192,arlit,
223,tanout,
271,kidal,
315,mecca,
349,gunnedah,
466,nizwa,
540,sao desiderio,


In [40]:
# define variable for nearby hotel parameters
locations = vacation_weather[["Latitude", "Longitude"]]

# Loop through dataframe and add nearby hotel result

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": locations,  
    "rankby": "distance",
    "type": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():

    # assemble url and make API request
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "n/a"
        
hotel_df       

IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map



# Display figure

